---

## Weeks 10 & 11 - Predicting Continuous Target Variables with Regression Analysis {-}


### Unit Convenor & Lecturer {-}
[George Milunovich](https://www.georgemilunovich.com)  
[george.milunovich@mq.edu.au](mailto:george.milunovich@mq.edu.au)

### References {-}

1. Python Machine Learning 3rd Edition by Raschka & Mirjalili - Chapter 10
2. Various open-source material

## Overview {-}

- Introducing Regression  
  - Simple Linear Regression  
  - Multiple Linear Regression  
- Exploring the Housing Dataset  
  - Visualizing the Important Characteristics of a Dataset  
- Implementing an Ordinary Least Squares Linear Regression Model  
  - Estimating Regression Parameters with Gradient Descent  
  - Estimating the coefficient of a regression model via scikit-learn  
- Fitting a Robust Regression Model using RANSAC  
- Evaluating the Performance of Linear Regression Models  
- Using Regularized Methods for Regression  
- Turning a Linear Regression Model Into a Curve - Polynomial Regression  
  - Modeling Nonlinear Relationships in the Housing Dataset  
  - Dealing with Nonlinear Relationships using Random Forests  
    - Decision Tree Regression  
    - Random Forest Regression  

---

**Revision: Supervised Learning**  
- *Classification*: a prediction problem where a class label is predicted for a given example of input data.   
    - E.g. predict if a mortgage customer will default on a mortgage payment or not 
    - Classify if an email is spam or not.  
- *Regression Analysis*: a prediction problem where the predicted target variable is continuous  
    - E.g. predict house prices on the basis of the house characteristics such as the number of bedrooms, bathrooms, etc.   
    
## Introducing Linear Regression {-}  

<br>

**Linear Regression**  
- Model the linear relationship between one or multiple features and a **numeric target variable**  
- Predict outputs on a continuous scale rather than categorical class labels  
- Model Training: learn regression parameters (weights) from the training dataset which will be used to make a prediction $\hat{y}$ 

<br>

**Simple (Univariate) Linear Regression**  


Model the relationship between a single feature (explanatory variable $x$) and a continuous target $y$  

True values of the target variable: $y= w_0 + w_1x + u$   
Predictions of the target variable: $\hat{y}=w_0 + w_1x$    
Prediction Errors (residuals) are computed as $u = y - \hat{y}$     

- $w_0$ - intercept  
- $w_1$ - regression slope / weight coefficient of the explanatory variable   
- $u$ = residual  

<img src="images/10_01.png" alt="Drawing" style="width: 400px;"/>  



# Multiple Linear Regression {-}

- Multiple Linear Regression: A linear regression with multiple explanatory variables  
- E.g Multiple linear regression with two exlanatory variables, $k=2$  

$y = w_0 x_0 + w_1x_1 + \dots + w_kx_k \quad$        where we set $x_0=1$  

<img src="images/10_16.png" alt="Drawing" style="width: 400px;"/>  

---

# Exploring the Housing Dataset {-}  

- House prices and 13 house attributes from the 1970s 



- Target: median value of owner-occupied homes in $1000s  

- Columns  

1. CRIM      per capita crime rate by town  
2. ZN        proportion of residential land zoned for lots over 25,000 sq.ft.  
3. INDUS     proportion of non-retail business acres per town  
4. CHAS      Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)  
5. NOX       nitric oxides concentration (parts per 10 million)  
6. RM        average number of rooms per dwelling  
7. AGE       proportion of owner-occupied units built prior to 1940  
8. DIS       weighted distances to five Boston employment centres  
9. RAD       index of accessibility to radial highways  
10. TAX      full-value property-tax rate per \$10,000  
11. PTRATIO  pupil-teacher ratio by town  
12. LSTAT    Percentage of low income residents  
13. MEDV     Median value of owner-occupied homes in \$1000s  


<hr style="width:35%;margin-left:0;">   

```


import pandas as pd
import numpy as np

df = pd.read_csv('data/Housing.csv')

df.head()

df.info()



```

<hr style="width:35%;margin-left:0;">   

## EDA - Exploring the Important Characteristics of a Dataset {-} 

- **Exploratory Data Analysis (EDA)**
    - EDA refers to analysing data to summarise their main characteristics, often using visual methods.
    - EDA helps to uncover patterns, relationships, anomalies, and other insights within the data.
    - It involves techniques like data visualisation, descriptive statistics, and sometimes simple modelling.
    - **EDA is often a preliminary step before more formal statistical techniques / analytics are applied and can be crucial for understanding the underlying structure of data sets.** 

- **Some Common EDA Techniques**
    - Descriptive Statistics Tables - provide basic summary statistics, e.g. mean, variance, min, max, etc.
    - Histograms (numeric data) / Pie Charts (categorical data) - a histogram is a graphical representation of the distribution of numerical data. 
    - Scatter Plots - display the relationship between two variables.
    - Correlation plots/tables/matrices - are a tabular representation of the correlation coefficients between pairs of variables in a data set.|
- Lets consider the basic summary statistics and some histograms first


1. Summary Statistics
```
pd.set_option('display.float_format', '{:.2f}'.format)
df.describe()
```




2. Histogram


```
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(10, 4))  

sns.histplot(data=df, x='MEDV', stat='percent', ax=axes[0])  
axes[0].set_title('Histogram for MEDV')

sns.histplot(data=df, x='RM', stat='percent',ax=axes[1])  
axes[1].set_title('Histogram for RM')

sns.histplot(data=df, x='LSTAT', stat='percent', ax=axes[2])  
axes[2].set_title('Histogram for LSTAT')

# Display the plot
plt.tight_layout()
plt.show()
```

3. Next we'll consider scatter plots between all pairs of variables in the dataset
    - A scatter plot is a type of data visualisation that displays individual data points as dots on a two-dimensional coordinate system
    - Each dot represents the value of two variables—one plotted along the horizontal axis (x-axis) and the other plotted along the vertical axis (y-axis)
 
```
import matplotlib.pyplot as plt

import seaborn as sns

sns.pairplot(df)

plt.show()

``` 



4. **Correlation**
- Linear correlation, also known as Pearson correlation, measures the strength and direction of the linear relationship between two continuous variables.
- The key concept here is that of **linear** relationship -> it may not capture nonlinear relationships
- $r=\frac{\sigma_{xy}}{\sigma_x\sigma_y}\in[-1,1]$
- The closer the correlation coefficient is to 1 or -1, the stronger the linear relationship between the variables.
- A correlation coefficient closer to 0 indicates a weaker linear relationship.
- Direction of Correlation: The sign of the correlation coefficient indicates the direction of the relationship:
    - Positive $r$ indicates a positive correlation: as one variable increases, the other variable tends to increase.
    - Negative $r$ indicates a negative correlation: as one variable increases, the other variable tends to decrease.
    - $r=-1$ -> perfect negative correlation  
    - $r=0$ -> no linear relationship  
    - $r=1$ -> perfect positive correlation  
    - Note: the covariance between standardized features is actually equal to their correlation coefficient  

5. **Correlation Matrix**  
    - Plot all correlations in one graph  
    - Look for high (both positive and negative) correlations  
    - From last row we see  
        - High positive (r=0.7) correlation between RM and MEDV and
        - High negative (r=-0.74) correlation between LSTAT and MEDV (but from scatterplot this relationship looks nonlinear)   
    - Use `corr()` from `pandas` to get correlation matrix, plot it using `heatmap` from `seaborn`  
    
```
df.corr()
```

- We can also plot the correlation matrix inside a heat map
    - Very light colours represent large positive correlations
    - Very dark colours represent large negative correlations
- Both large positive and large negative can be used in predictive models

```
import seaborn as sns

corrmat = df.corr()
# print(corrmat.round(3).to_string())
# corrmat

f, ax = plt.subplots(figsize=(20, 10))
sns.heatmap(corrmat, annot=True, vmax=.8, square=True)
plt.show()
```

<br>
<br>

---  

## Implementing an Ordinary Least Squares Linear Regression Model {-}  

- Linear regression computes the equation of the best-fitting straight line through the examples of the dataset  
- How is 'best-fitting' defined?  
    - Ordinary Least Squares (OLS) - estimate parameters of the linear regression line which *minimises* the sum of squared verticial distances (prediction errors/residuals) from the estimated line to the training examples   

        

- OLS implements the same cost function (of predictions errors) that Adaline employs to estimate its weights  
    - Sum of Squared Errors (SSE) cost function: $J(w)=\frac{1}{2}\sum_{i=1}^n\left(y_{i} - \hat{y}_{i}\right)^2$  
    - Given that that cost function is **exactly the same as in Adaline** we can optimize regression weights (betas) using gradient descent (GD) or stochastic gradient descent (SGD) as in Adaline  
        - A gradient descent implementation of OLS regression is provided in the textbook  
    - Besides GD and similar iterative optimisation algorithms there is a closed form solution for solving OLS parameters  
        - $w=(X^{'}X)^{-1}X^{'}y$ making sure that the matrix $X$ has a column of 'ones' (usually first column)  
        - This solution for $w$ does not require standardising the data  
    - Regression predictions are formed as $\hat{y}=w_0x_0 + w_1x_1 + w_2x_2 + \dots + w_kx_k = \sum_{i=1}^k w_ix_i$  
        - OLS regression is essentially equal to Adaline but without the unit step function so that we obtain continuous predictions of the target variable (rather than class labels -1 and 1)  

<hr style="width:35%;margin-left:0;"> 

<span style='background:orange'>  **Example: Closed-form solution for simple linear regression**    

1. Add a column of 'ones' to df  
2. Set 'MEDV' - median house price value - as $y$, ['ones', 'RM'], where RM is the number of rooms as $X$  
3. Compute the weights (betas) of the linear regression $\hat{y}=w_0 + w_1x$ using the following formula $w=(X^{'}X)^{-1}X^{'}y$  
4. Compute the fitted values (predictions) of $y$ given $X$ using the formula $\hat{y}=Xw$ and plot the prediction together with the scatterplot of RM and MEDV  
5. Predict house price for a house with 5 rooms using the formula $𝑦̂=𝑤′𝑥$  

<hr style="width:35%;margin-left:0;"> 

1. Add a column of 'ones' to df

```
df['ones'] = 1
df
```

<hr style="width:35%;margin-left:0;"> 

2. Set 'MEDV' - median house price value - as $y$, ['ones', 'RM'], where RM is the number of rooms as $X$

```
X = df[['ones', 'RM']].values
y = df['MEDV'].values

# X
# y

```

<hr style="width:35%;margin-left:0;"> 

3. Compute the weights (betas) of the linear regression $\hat{y}=w_0 + w_1x$ using the following formula $w=(X^{'}X)^{-1}(X^{'}y)$

```
w = np.linalg.inv(X.T@X)@(X.T@y)

print(w)

print(f'Intercept: {w[0]:.3f}')
print(f'Slope: {w[1]:.3f}')
```

<hr style="width:35%;margin-left:0;">   

4. Compute the fitted values (predictions) of $y$ given $X$ using the formula $\hat{y}=w_0x_0+w_1x_1=Xw$ and plot the prediction together with the scatterplot of RM and MEDV

```
import matplotlib.pyplot as plt

y_hat = X@w

plt.scatter(X[:,1], y, c = 'steelblue', edgecolor='white', s=70)
plt.plot(X[:,1], y_hat, color = 'black', lw = 2)

plt.xlabel('Average number of rooms [RM]')
plt.ylabel('Price in $1,000 [MEDV]')
plt.show()
```

<hr style="width:35%;margin-left:0;">   

5. Predict house price for a house with 5 rooms using the formula $\hat{y}=w_0x_0+w_1x_1= w^{'}x$

```
x_for_prediction = [1, 5]

y_hat_x_5 = w.T@x_for_prediction
print(f'Predicted price (in $ thousands) for X = 5 rooms: {y_hat_x_5:.3f}')
```

<hr style="width:35%;margin-left:0;">   

<span style='background:orange'>  **Example: Simple linear regression with scikit-learn**   
    
1. Compute the weights (betas) of the linear regression $\hat{y}=w_0 + w_1x$ using the `fit` method of `LinearRegression`  
2. Compute the fitted values (predictions) of $y$ given $X$ using the `predict` method and plot the prediction together with the scatterplot of RM and MEDV   
3. Predict house price for a house with 5 rooms using `predict` method  

<hr style="width:35%;margin-left:0;">   

1. Compute the weights (betas) of the linear regression $\hat{y}=w_0 + w_1x$ using the `fit` method of `LinearRegression`

```
from sklearn.linear_model import LinearRegression

slr = LinearRegression()

slr.fit(X=df[['RM']], y=df['MEDV'])

print(f'Intercept: {slr.intercept_:.3f}')
print(f'Slope: {slr.coef_[0]:.3f}')
```

<hr style="width:35%;margin-left:0;">  

2. Compute the fitted values (predictions) of $y$ given $X$ using the `predict` method and plot the prediction together with the scatterplot of RM and MEDV

```
y_predict = slr.predict(df[['RM']])
# y = df['MEDV']

plt.scatter(df[['RM']], df['MEDV'], c = 'steelblue', edgecolor='white', s=70)
plt.plot(df[['RM']], y_predict, color = 'black', lw = 2)

# plt.scatter(X[:,1], y, c = 'steelblue', edgecolor='white', s=70)
# plt.plot(X[:,1], y_predict, color = 'black', lw = 2)

plt.xlabel('Average number of rooms [RM]')
plt.ylabel('Price in $1,000 [MEDV]')
plt.show()

```

<hr style="width:35%;margin-left:0;">  

3. Predict house price for a house with 5 rooms using `predict` method

```
# X_predict = np.array([5]) # 1-d array doesn't work with sklearn
# X_predict = np.array([5]).reshape(-1,1)  # 1st way of reshaping into a 2-d array
# X_predict = np.array([[5]]) # 2nd way of reshaping

X_predict = pd.DataFrame({'RM': [5]})

y_hat_x_5_sklearn = slr.predict(X_predict)
print(f'Predicted price (in $ thousands) for X = 5 rooms: {y_hat_x_5_sklearn[0]:.3f}')
```

---

# Fitting a Robust Regression Model using RANSAC {-}  

**Outliers**: data points that differ significantly from other observations.   
- An **outlier** may represent a true low probability event, or it may indicate a measurement error  
- Regression models can be heavily impacted by the presence of outliers  
    - This is a problem since regression models are meant to capture relationships which hold on average, and outliers, by definition, are far away from the average
    - Thus, outliers can affect the results and statistical analyses significantly, often skewing the data disproportionately     
- There are many statistical tests to detect outliers  
    - Once detected outliers are sometimes removed from the dataset  
- Instead of directly removing outliers we will implement the RANdom SAmple Consensus (RANSAC) algorithm  
    - Fit a regression model to a subset of data, the so-called **Inliers**  
    - [https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RANSACRegressor.html](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RANSACRegressor.html)

<hr style="width:35%;margin-left:0;">   

**RANSAC Algorithm**  
- `from sklearn.linear_model import RANSACRegressor`  
    - Set the smallest number of examples to be used in estimation
    - Select allowed distance from the fitted line, and 
    - Expand the initial set with consistent data points  


1. Select a minimum number of randomly chosen examples to be treated as inliers and fit the model  
    - E.g. set `min_samples = 50`  
2. Test all other data points against the fitted model and add those points which are within a user-given distance from the fitted line  
    - E.g. set the distance to 5  (`residual_threshold=5`) - Maximum residual for a data sample to be classified as an inlier  
    - This value is problem specific and while 5 works in our problem it may not always work. This is a disadvantage of RANSAC algorithm  
3. Refit the model using all inliers  
4. Terminate the algorithm if the fraction of the number of inliers over the sample size exceeds a predefined threshold or if a fixed number of iterations are reached. Go to step 1 otherwise.  
    - max_trials: after all trials, the model with the highest number of inliers is selected.


<hr style="width:35%;margin-left:0;">   

<span style='background:orange'>  **Example: Fit linear regression model with the RANSAC algorithm**   

1. Fit a linear regression via RANSAC with 'MEDV' - median house price value - as $𝑦$, 'RM' as $x$  
2. Plot the fitted line and label datapoints as inliers (use `inliear_mask_`) or outliers  
3. Print the slope and intercept of the estimated (robust) regression line  

<hr style="width:35%;margin-left:0;">   

1. Fit a linear regression via RANSAC with 'MEDV' - median house price value - as $𝑦$, 'RM' as $x$

```
from sklearn.linear_model import RANSACRegressor

X = df[['RM']]
y = df['MEDV']

ransac = RANSACRegressor(LinearRegression(), 
                         max_trials=100, 
                         min_samples=50, 
                         residual_threshold=5.0,   # example an inlier if distance from the fitted line within 5 units
                         random_state=0)

ransac.fit(X, y)

```

<hr style="width:35%;margin-left:0;">   

2. Plot the fitted line and label datapoints as inliers (use `inliear_mask_`) or outliers

```
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

# print(inlier_mask[inlier_mask == True].shape)
# print(pd.DataFrame(np.vstack((inlier_mask, outlier_mask)).T))
# print(pd.DataFrame(np.vstack((inlier_mask, outlier_mask)).T).value_counts())

line_X = np.arange(3, 10, 1)

line_y_ransac = ransac.predict(line_X.reshape(-1,1))

plt.scatter(X[inlier_mask], y[inlier_mask], c='steelblue', edgecolor='white', marker='o', label='Inliers')
plt.scatter(X[outlier_mask], y[outlier_mask], c='limegreen', edgecolor='white', marker='s', label='Outliers')
plt.plot(line_X, line_y_ransac, color='black', lw=2)   

plt.xlabel('Average number of rooms [RM]')
plt.ylabel('Price in $1000s [MEDV]')
plt.legend(loc='upper left')

#plt.savefig('images/10_08.png', dpi=300)
plt.show()
```

<hr style="width:35%;margin-left:0;">   

3. Print the slope and intercept of the estimated (robust) regression line

```
print(f'Slope: {ransac.estimator_.coef_[0]:.3f}')
print(f'Intercept: {ransac.estimator_.intercept_:.3f}')
```

---

for the entire dataset

Slope: 9.102
Intercept: -34.671

---

## Evaluating the Performance of Linear Regression Models {-}  

  
1. **Mean Squared Error (MSE)**  
- $MSE=\frac{1}{n}\sum_{i=1}^n\left(y_{i} - \hat{y}_{i}\right)^2$   
- MSE = average of squared prediction errors
- Objective: **Minimise MSE** (build a model to make it as small as possible)
- Interpretation depends on feature scaling/units of measurement  
- Can be computed both on:  
    - Training dataset -> in order to tune hyperparameters and compare/rank different models 
    - Test dataset -> evaluate forecasting performance, see if the model generalises well, and compare/rank different models  
    - If the MSE computed on the training dataset is much smaller than the MSE computed from test examples -> sign of overfitting  
  
2. **Coefficient of Determination $(R^2)$**  
- $R^2 = 1 - \frac{\sigma_u^2}{\sigma_y^2}=r_{y,\hat{y}}^2\in[0,1]$  
- If $R^2=1\Rightarrow MSE=0\Rightarrow$Perfect Fit

3. **Residual Plot** - plot the **Residuals** (y-axis) vs. **Predicted Values** (x-axis)  
- If a model provides a good fit then Residuals (errors) should be:  
    - Randomly distributed around the centre line  
    - No outliers with large deviations from the centre line  
- Indicators of potential problems  
    - Patterns in residual plots -> model unable to capture some explanatory features (potential missing variables or nonlinearity) 
    - Large outliers  

<hr style="width:35%;margin-left:0;">   


<span style='background:orange'>  **Example: Evaluating Regression Performance**   
    
1. Use all house features as X and set y = MEDV. Split data into train and test datasets with test size being 30% of the total sample size.  
2. Fit a linear regression to training data, and predict house prices for both training and test datasets  
3. Compute MSE and $R^2$ for the train and test datasets  
4. Graph the residual plot for both training and test datasets  


<hr style="width:35%;margin-left:0;">   

1. Use all house features as X and set y = MEDV. Split data into train and test datasets with test size being 30% of the total sample size.

```
from sklearn.model_selection import train_test_split

y = df['MEDV']
X = df.drop(columns = ['MEDV', 'ones'])

X

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


```

<hr style="width:35%;margin-left:0;">   

2. Fit a linear regression to training data, and predict house prices for both training and test datasets

```
slr = LinearRegression()

slr.fit(X_train, y_train)

y_train_pred = slr.predict(X_train)
y_test_pred = slr.predict(X_test)
```

<hr style="width:35%;margin-left:0;">   

3. Compute MSE and $R^2$ for the training and test datasets

```
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

print(f'MSE train: {mean_squared_error(y_train, y_train_pred):.3f}, test: {mean_squared_error(y_test, y_test_pred):.3f}')
print(f'R^2 train: {r2_score(y_train, y_train_pred):.3f}, test: {r2_score(y_test, y_test_pred):.3f}')
```

<hr style="width:35%;margin-left:0;">   

3. Graph the residual plot for both training and test datasets

```
plt.scatter(y_train_pred,  y_train_pred - y_train, c='steelblue', marker='o', edgecolor='white', label='Training data')
plt.scatter(y_test_pred,  y_test_pred - y_test, c='limegreen', marker='s', edgecolor='white', label='Test data')
plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=-10, xmax=50, color='black', lw=2)
plt.xlim([-10, 50])
plt.tight_layout()

# plt.savefig('images/10_09.png', dpi=300)
plt.show()
```

---

## Using Regularized Methods for Regression {-}  

- In regression analysis, like in classification, we often face the problem of overfitting  
    - Regularisation: introduce a penalty against complexity by shrinking parameter values  
    - Regularisation methods in regression analysis:  
        - L2 regularisation: Ridge Regression  
        - L1 regularisation: LASSO (Least Absolute Shrinkage and Selection Operator)  
        - L1 + L2 regularisation: Elastic Net  
         
**Ridge Regression**  
- Ridge Regression is just an L2 penalised model that we have seen before  
- We modify the least-squares cost function as follows $J(w)=\sum_{i=1}^n\left(y_{i} - \hat{y}_{i}\right)^2 + \lambda||w||_2^2$   
    - As before $\lambda||w||_2^2=\lambda\sum_{j=1}^mw_j^2$  
    - Hyperparameter $\lambda$ - strength of regularization $(\lambda\uparrow\Rightarrow$Regularization$\uparrow)$   
    - Note: do not regularize the intercept $w_0$  
    
**LASSO (Least Absolute Shrinkage and Selection Operator)**  
- L1 regularization  
- $J(w)=\sum_{i=1}^n\left(y_{i} - \hat{y}_{i}\right)^2 + \lambda||w||_1$   
    - $\lambda||w||_1=\lambda\sum_{j=1}^m|w_j|$  
- When regularization strength is high-> certain weights can become zero -> LASSO = supervised feature-selection technique  
- Limitation: selects at most $n$ features when $m>n$   
    - $m=$ number of features, $n=$ is the number of observations
    - However this also avoids saturation (when n = m -> saturated model -> fits data perfectly but does not generalise well)  
    
**Elastic Net**  
- Both L1 and L2 penalties  
- $J(w)=\sum_{i=1}^n\left(y_{i} - \hat{y}_{i}\right)^2 + \lambda_1||w||_2^2 + \lambda_2||w||_1$   
    - L1 penalty generates sparsity (feature selection)  
    - L2 penalty allows for selecting more than $n$ features when $m>n$  
    
    
**Implementations in scikit-learn**  
- Use similarly to `LinearRegression`  
    - `from sklearn.linear_model import Ridge`  
    - `from sklearn.linear_model import Lasso`  
    - `from sklearn.linear_model import ElasticNet`  
- Must specify $\lambda_1$ and/or $\lambda_2$  
    - These are most often optimised via k-fold cross-validation   

<hr style="width:35%;margin-left:0;">   

<span style='background:orange'>  **Example: Regularized Regression Models**   
    
1. Using the same data as in the last exercise above fit a LASSO regression to training data, and predict house prices for both training and test datasets  
2. Compute $R^2$ and MSE for both train and test datasets and print estimated LASSO coefficients  
3. Repeat 1. and 2 using a Ridge regression  
4. Repeat 1. and 2 using an Elastic Net regression  

<hr style="width:35%;margin-left:0;">  

1. Using the same data as in the last exercise above fit a LASSO regression to training data, and predict house prices for both training and test datasets

```
from sklearn.linear_model import Lasso
    
lasso = Lasso(alpha=0.1) # alpha = lambda (above)

lasso.fit(X_train, y_train)

y_train_pred = lasso.predict(X_train)
y_test_pred = lasso.predict(X_test)
```

<hr style="width:35%;margin-left:0;">  

2. Compute $R^2$ and MSE for both train and test datasets and print estimated LASSO coefficients

```
np.set_printoptions(precision=3, suppress = True)       # format printing to 3 decimal places in numpy


print(f'LASSO MSE train: {mean_squared_error(y_train, y_train_pred):.3f}, test: {mean_squared_error(y_test, y_test_pred):.3f}')
print(f'LASSO R^2 train: {r2_score(y_train, y_train_pred):.3f}, test: {r2_score(y_test, y_test_pred):.3f}\n')

print(f'LASSO coefficients:\n {lasso.coef_}')
print(f'LASSO intercept:\n {lasso.intercept_}')
```

<hr style="width:35%;margin-left:0;">  

3. Repeat 1. and 2 using a Ridge regression

```
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=0.1)

ridge.fit(X_train, y_train)

y_train_pred_ridge = ridge.predict(X_train)
y_test_pred_ridge = ridge.predict(X_test)

print(f'Ridge MSE train: {mean_squared_error(y_train, y_train_pred_ridge):.3f}, test: {mean_squared_error(y_test, y_test_pred_ridge):.3f}')
print(f'Ridge R^2 train: {r2_score(y_train, y_train_pred_ridge):.3f}, test: {r2_score(y_test, y_test_pred_ridge):.3f}\n')

print(f'Ridge coefficients:\n {ridge.coef_}')
```

<hr style="width:35%;margin-left:0;">  

4. Repeat 1. and 2 using an Elastic Net regression

```
from sklearn.linear_model import ElasticNet
elasticnet = ElasticNet(alpha=0.001, l1_ratio=0.5)

elasticnet.fit(X_train, y_train)
y_train_pred_elasticnet = elasticnet.predict(X_train)
y_test_pred_elasticnet = elasticnet.predict(X_test)

print(f'Elastic Net MSE train: {mean_squared_error(y_train, y_train_pred_elasticnet):.3f}, test: {mean_squared_error(y_test, y_test_pred_elasticnet):.3f}')
print(f'Elastic Net R^2 train: {r2_score(y_train, y_train_pred_elasticnet):.3f}, test: {r2_score(y_test, y_test_pred_elasticnet):.3f}\n')

print(f'Elastic Net coefficients:\n {elasticnet.coef_}')
```

---

## Turning a Linear Regression Model into a Curve - Polynomial Regression {-}  

- So far we have assumed a linear relations between the target $y$ and features $x$  
- One way to account for nonlinear patterns is to use a polynomial regression  
    - $y=w_0 + w_1x + w_2x^2 + \dots + w_dx^d$  
        - Here $d$ denotes the degree of the polynomial  
        - Note: adding polynomial features increases the complexity of the model which raises the chance of overfitting, and is the reason why we use kernel methods in the first place  
    - `from sklearn.preprocessing import PolynomialFeatures`   
- An alternative to polynomial features is a log-transformation  
    - Either just transform $x$, or both $x$ and $y$  
        - E.g. $y=w_0 + w_1 log(x)$ or  
        - E.g. $log(y) = w_0 + w_1 log(x)$  -> must convert back to $y$ after making the predictions using $\hat{y}=exp(log(\hat{y}))$


<hr style="width:35%;margin-left:0;">   

<span style='background:orange'>  **Example: Regressions with Polynomial Features**   

1. Generate nonlinear data and plot it 
2. Create a quadratic term of $x$ using `PolynomialFeatures`  
3. Fit a linear regression and a polynomial regression & plot fitted values vs true data  
4. Predict $y$ using training dataset for X, compare MSE and $R^2$ across the linear and quadratic regressions  


<hr style="width:35%;margin-left:0;">   

1. Generate nonlinear data and plot it 

```
import seaborn as sns

X = np.array([258.0, 270.0, 294.0, 320.0, 342.0, 368.0, 396.0, 446.0, 480.0, 586.0]).reshape(-1,1)
y = np.array([236.4, 234.4, 252.8, 298.6, 314.2, 342.2, 360.8, 368.0, 391.2, 390.8])

df2 = pd.DataFrame(np.hstack((y.reshape(-1,1), X)), columns = ['y', 'X'])
print(df2)

sns.regplot(data = df2, x='X', y='y', ci = None) 

plt.show()
```

<hr style="width:35%;margin-left:0;">   
2. Create a quadratic term of $x$ using `PolynomialFeatures`

```
from sklearn.preprocessing import PolynomialFeatures

quadratic = PolynomialFeatures(degree=2)
X_quad = quadratic.fit_transform(X)

X_quad
```

<hr style="width:35%;margin-left:0;">   

3. Fit a linear regression and a polynomial regression & plot fitted values vs true data

```
lr_1 = LinearRegression()
lr_2 = LinearRegression()

# ----- fit linear features
lr_1.fit(X, y)
y_lin_fit = lr_1.predict(X)

# ----- fit quadratic features
lr_2.fit(X_quad, y)
y_quad_fit = lr_2.predict(X_quad)

# plot fitted values
plt.scatter(X, y, label='Training points')
plt.plot(X, y_lin_fit, label='Linear fit', linestyle='--')
plt.plot(X, y_quad_fit, label='Quadratic fit')

plt.xlabel('Explanatory variable')
plt.ylabel('Predicted or known target values')
plt.legend(loc='upper left')

plt.tight_layout()
#plt.savefig('images/10_11.png', dpi=300)
plt.show()
```

<hr style="width:35%;margin-left:0;">   

4. Predict $y$ using training dataset for X, compare MSE and $R^2$ across the linear and quadratic regressions

```
y_lin_pred = lr_1.predict(X)
y_quad_pred = lr_2.predict(X_quad)

print(f'Training MSE linear: {mean_squared_error(y, y_lin_pred):.3f}, quadratic: {mean_squared_error(y, y_quad_pred):.3f}')
print(f'Training R^2 linear: {r2_score(y, y_lin_pred):.3f}, quadratic: {r2_score(y, y_quad_pred):.3f}')
```

<hr style="width:35%;margin-left:0;">   

### Modeling Nonlinear Relationships in the Housing Dataset

<hr style="width:35%;margin-left:0;">   

<span style='background:orange'>  **Example: Nonlinear Relationships in the Housing Dataset** 
1. Model the relationship between MEDV (medium house price value) and LSTAT (% of low income population) using 2nd (quadratic) and 3rd (cubic) degree polynomials
    
```


X = df[['LSTAT']].values
y = df['MEDV'].values

regr = LinearRegression()

# ---- create quadratic features
quadratic = PolynomialFeatures(degree=2)
X_quad = quadratic.fit_transform(X)
# print(X_quad)

```


```
# ---- create cubic features
cubic = PolynomialFeatures(degree=3)
X_cubic = cubic.fit_transform(X)
# print(X_cubic)

# create X for plotting
X_fit = np.arange(X.min(), X.max(), 1).reshape(-1, 1) # [:, np.newaxis]
# X_fit = np.arange(X.min(), X.max(), 1)[:, np.newaxis]
```

```
# fit learn regression
regr = regr.fit(X, y)
y_lin_fit = regr.predict(X_fit)
linear_r2 = r2_score(y, regr.predict(X))

# fit quadradit regression
regr = regr.fit(X_quad, y)
y_quad_fit = regr.predict(quadratic.fit_transform(X_fit))
quadratic_r2 = r2_score(y, regr.predict(X_quad))

# fit cubic regression
regr = regr.fit(X_cubic, y)
y_cubic_fit = regr.predict(cubic.fit_transform(X_fit))
cubic_r2 = r2_score(y, regr.predict(X_cubic))
```

```
# plot results
plt.scatter(X, y, label='Training points', color='lightgray')

plt.plot(X_fit, y_lin_fit, label=f'Linear (d=1), $R^2={linear_r2:.2f}$', color='blue', lw=2, linestyle=':')
plt.plot(X_fit, y_quad_fit, label=f'Quadratic (d=2), $R^2={quadratic_r2:.2f}$', color='red', lw=2, linestyle='-')
plt.plot(X_fit, y_cubic_fit, label=f'Cubic (d=3), $R^2={cubic_r2:.2f}$', color='green', lw=2, linestyle='--')

plt.xlabel('% lower status of the population [LSTAT]')
plt.ylabel('Price in $1000s [MEDV]')
plt.legend(loc='upper right')

# #plt.savefig('images/10_12.png', dpi=300)
plt.show()
```

---

## Dealing with Nonlinear Relationships using Decision Trees and Random Forests {-}

- Decision Tree *classifiers* we considered previously can be modified to form decision tree *regressions*  

  
   
- **Decision Tree Regression**  
    - Can fit nonlinear features without feature transformation   
    - Decision trees analyse one feature at a time  
    - Iteratively split nodes until the leaves are pure or a stopping criterion is satisfied   
    - In order to measure impurity at node $t$ need an impurity metric that is suitable for continuous variables  
        - Use MSE - also known as **within-node variance**  
        - When using MSE as an impurity measure the splitting criterion is known as **variance reduction**  
    - $I(t)=MSE(t)=\frac{1}{N_t}\sum_{i\in D_t}(y_{i}-\hat{y}_t)^2$  
        - $N_t$ - number of training examples at node $t$  
        - $D_t$ - training subset at node $t$  
        - $\hat{y}_t=\frac{1}{N_t}\sum_{i\in D_t}y_{i}$  
    - Need to choose an appropriate depth of the tree so as not to overfit or underfit  
                 

- **Random Forest Regression**  
    - Ensemble technique combining multiple decision trees  
        - Use MSE criterion to grow individual trees  
        - Predicted target variable - average prediction over all decision trees  
    - Advantages over decision trees  
        - Usually better generalization performance (test sample predictions) than an individual decision trees  
        - Don't require much parameter tuning  
            - Still need to tune number of decision trees  

<hr style="width:35%;margin-left:0;">   

<span style='background:orange'>  **Example: Nonlinear Relationships with Decision Trees and Random Forests**   
1. Fit a decision tree regression on MEDV (medium house price value) and LSTAT (% of lower status population) and plot fitted values  
2. Split the data into 60% train and 40% test datasets  
3. Fit a random forest regression on on MEDV (medium house price value) and all available features in the dataset. Print train and test $R^2$ and MSEs.  
4. Plot train and test set residuals      

<hr style="width:35%;margin-left:0;">  

1. Fit a decision tree regression on MEDV (medium house price value) and LSTAT (% of lower status population) and plot fitted values

```
from sklearn.tree import DecisionTreeRegressor

X = df[['LSTAT']].values
y = df['MEDV'].values

tree = DecisionTreeRegressor(max_depth=3)

tree.fit(X, y)


plt.scatter(X, y, c='steelblue', edgecolor='white', s=70)

sort_idx = X.flatten().argsort()
plt.plot(X[sort_idx], tree.predict(X[sort_idx]), c='black', lw=2)

plt.xlabel('% lower status of the population [LSTAT]')
plt.ylabel('Price in $1000s [MEDV]')
#plt.savefig('images/10_14.png', dpi=300)
plt.show()
```

<hr style="width:35%;margin-left:0;">  

2. Split the data into 60% train and 40% test datasets

```


X = df.drop(columns = ['MEDV', 'ones'])
y = df['MEDV']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

X_train


```


<hr style="width:35%;margin-left:0;">  

3. Fit a random forest regression on on MEDV (medium house price value) and all available features in the dataset. Print train and test $R^2$ and MSEs.

```
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=1000, 
                               criterion='squared_error', 
                               random_state=1, 
                               n_jobs=-1)
forest.fit(X_train, y_train)

y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

print(f'MSE train: {mean_squared_error(y_train, y_train_pred):.3f}, test: {mean_squared_error(y_test, y_test_pred):.3f}')
print(f'R^2 train: {r2_score(y_train, y_train_pred):.3f}, test: {r2_score(y_test, y_test_pred):.3f}')
```

<hr style="width:35%;margin-left:0;">  

4. Plot train and test set residuals

```
plt.scatter(y_train_pred, y_train_pred - y_train, c='steelblue', edgecolor='white', marker='o', s=35, alpha=0.9, label='Training data')
plt.scatter(y_test_pred, y_test_pred - y_test, c='limegreen', edgecolor='white', marker='s', s=35, alpha=0.9, label='Test data')

plt.xlabel('Predicted values')
plt.ylabel('Residuals')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=-10, xmax=50, lw=2, color='black')
plt.xlim([-10, 50])
plt.tight_layout()

#plt.savefig('images/10_15.png', dpi=300)
plt.show()
```

---

## Other Nonlinear Methods {-}

Most classifiers that we have considered so far have their regression counterparts that you can experiment with on your own.

- Use Kernel PCA to extract nonlinear factors to be used in a linear regression  
- `from sklearn.ensemble import BaggingRegressor` - Bagging Regressor  
- `from sklearn.ensemble import AdaBoostRegressor` - AdaBoost Regressor  
- `from sklearn.svm import SVR` - Support Vector Regressor  
- `from sklearn.neural_network import MLPRegressor` - Multi-layer Perceptron (Neural Network) Regressor  
- Etc.  

## Forecast Combinations - Ensemble Methods

In addition to the algorithms metioned above we should also explore combining the forecasts from several regressions via

- Voting 
    - A voting regressor is an ensemble meta-estimator that fits several base regressors, each on the whole dataset. 
    - Then it averages the individual predictions to form a final prediction.
    - [https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingRegressor.html](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingRegressor.html)
- Stacking
    - Stacking combines forecasts from different regressions by using the predictions from each base models as a new feature
    - The architecture of a stacking model involves two or more base models, often referred to as level-0 models, and a meta-model that combines the predictions of the base models, referred to as a level-1 model.
    - Level-0 Models (Base-Models): Models fit on the training data and whose predictions are compiled.
    - Level-1 Model (Meta-Model): Model that learns how to best combine the predictions of the base models.
    - [https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.StackingRegressor.html)
    
    
```
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


estimators = [('ridge', RidgeCV()), ('svr', LinearSVR())]
               # , 
               # ('linear_reg', LinearRegression(random_state=1))]

stacking_reg = StackingRegressor(estimators=estimators, final_estimator = DecisionTreeRegressor(random_state = 1))

stacking_reg.fit(X_train_scaled, y_train)

```

```
y_train_pred_stacking = stacking_reg.predict(X_train_scaled)
y_test_pred_stacking = stacking_reg.predict(X_test_scaled)


print(f'MSE train: {mean_squared_error(y_train, y_train_pred_stacking):.3f}, test: {mean_squared_error(y_test, y_test_pred_stacking):.3f}')
print(f'R^2 train: {r2_score(y_train, y_train_pred):.3f}, test: {r2_score(y_test, y_test_pred):.3f}')
```

---

## Serializing Fitted Scikit-Learn Estimators {-}

- Training a machine learning model can be computationally expensive especially if we are doing a cross-validation grid search of hyperparameters
    - Don't want to re-train our model every time we close Python interpreter and want to make a new prediction or reload web application
- Use built-in `pickle` to *serialize* and *deserialize* Python objects so we can save our trained classifier (don't need to retrain it)
    - Serialization is the process of translating a data structure or object state from computer memory into a format that can be stored or transmitted and reconstructed later
    - `pickle.dump` - save an object to a file object
    - `pickle.load` - load an object from a file object
    - Warning: `pickle` can be a **security risk** as it executes code that has been stored in a pickle file. Do not unpickle files from unknown sources



- Refit the model as above

```
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators=1000, 
                               criterion='squared_error', 
                               random_state=1, 
                               n_jobs=-1)
forest.fit(X_train, y_train)

y_train_pred = forest.predict(X_train)
y_test_pred = forest.predict(X_test)

print(f'MSE train: {mean_squared_error(y_train, y_train_pred):.3f}, test: {mean_squared_error(y_test, y_test_pred):.3f}')

```


### Save a trained model

```
import pickle
pickle.dump(forest, open("forest.pkl", "wb"))
```




### Import a saved models

```
saved_forest = pickle.load(open("forest.pkl", 'rb'))
```


### Test the imported model

```
y_train_pred_saved = saved_forest.predict(X_train)
y_test_pred_saved = saved_forest.predict(X_test)

print(f'MSE train: {mean_squared_error(y_train, y_train_pred_saved):.3f}, test: {mean_squared_error(y_test, y_test_pred_saved):.3f}')

```